## 1, 사전 준비
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/recipekogpt2

/content/drive/MyDrive/recipekogpt2


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 808kB 5.1MB/s 
     |████████████████████████████████| 2.3MB 12.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 829kB 33.3MB/s 
     |████████████████████████████████| 645kB 42.7MB/s 
     |████████████████████████████████| 10.6MB 39.6MB/s 
     |████████████████████████████████| 122kB 55.0MB/s 
     |████████████████████████████████| 276kB 39.7MB/s 
     |████████████████████████████████| 3.3MB 19.0MB/s 
     |████████████████████████████████| 901kB 31.0MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 1.3MB 31.8MB/s 
     |████████████████████████████████| 296kB 41.3MB/s 
     |████████████████████████████████| 143kB 46.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=d72d274a152a4e19ba04f61937c36292a4edf1297d9d68cfd28722227b50c9ec
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dc

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 450kB 38.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


### 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.
downstream_model_dir에 정보를 입력하면 파인튜닝을 마친 모델로 인퍼런스를 합니다.

In [ ]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="models/model_checkpoints_l300_b16",
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/recipekogpt2/models/model_checkpoints_l300_b16/epoch=6-val_loss=1.68.ckpt


### 모델 로딩
프리트레인한(혹은 파인튜닝을 마친) GPT2 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
if args.downstream_model_checkpoint_fpath is None:
    model = GPT2LMHeadModel.from_pretrained(
        args.pretrained_model_name,
    )
else:
    pretrained_model_config = GPT2Config.from_pretrained(
        args.pretrained_model_name,
    )
    model = GPT2LMHeadModel(pretrained_model_config)
    fine_tuned_model_ckpt = torch.load(
        args.downstream_model_checkpoint_fpath,
        map_location=torch.device("cpu"),
    )
    model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


## 2. 학습 전 확인

토크나이저를 불러들이고, 미리 정의된 토큰을 추가해줍니다.

In [ ]:
tokens_list = ['<unused0>','<unused1>','<unused2>','<unused3>','<unused4>','<unused5>']
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', additional_special_tokens = tokens_list)

토큰화와 인코딩이 잘 되었는지 확인합니다.

In [ ]:
input_ids = tokenizer.encode('<unused0>연근 구기자 물김치<unused1><unused2>연근$손질$물$식초$구기자$물김치$동치미$국물$무$사과$배추$장$오이$돌$미나리$쪽파$<unused3><unused4>', return_tensors='pt')
input_ids

tensor([[    9, 33017,  9073, 20882,  9169,  6963,  8420,    10,    11, 33017,
           379,  7827,  8267,   379,  7561,   379,  7889,  8381,   379,  6919,
         20882,   379,  7561,  6963,  8420,   379,  7244,  8420,  7584,   379,
          6920,  7561,   379,  7556,   379,  7756,  6903,   379,  7609,  8397,
           379,  8168,   379,  8052,  8146,   379,  7239,   379,  7584, 20716,
           379,  8306,  8615,   379,    12,    13]])

In [ ]:
input_ids = tokenizer.encode('<unused0>비빔밥<unused1><unused2>콩나물$고추장$간장$상추$밥$계란$<unused3><unused4>',return_tensors='pt')
input_ids

tensor([[    9,  9072, 43933,    10,    11, 12213, 20551,   379, 22278,  8168,
           379,  6826,  8168,   379,  7767,  8397,   379,  7605,   379,  6886,
          7374,   379,    12,    13]])

In [ ]:
import pandas as pd
import pickle
with open('data/valdata.pickle','rb') as f:
  df = pickle.load(f)
df.head()

,input,output,split
0,<unused0>단호박 케일밥<unused1><unused2>단호박$찰밥$찹쌀$밤$...,찹쌀을 깨끗이 씻어 물에 30분 정도 불린다. 대추는 씨를 제거하고 돌려깎은 후 채...,"[단호박, 찰밥, 찹쌀, 밤, 은행, 잣, 대추, 케일, 쪄진, 물, 설탕]"
1,<unused0>치커리샐러드와 올리브 마늘 소스<unused1><unused2>치커...,마늘은 곱게 다진다. 올리브유 식초 설탕 다진 마늘을 섞어 거품기로 충분히 저어주 ...,"[치커리, 샐러드, 적양배추, 양파, 당근, 올리브마늘, 드레싱, 올리브유, 식초,..."
2,<unused0>시금치 우유 소스와 그린매쉬드포테이토<unused1><unused2...,시금치는 끓는 소금물에 데쳐 찬물에 헹구어 물기를 짜고 우유를 넣고 블렌더에 곱게 ...,"[그린, 매쉬드, 포테이토, 감자, 시금치우유, 아몬드, 설탕, 크랜베리, 치커리,..."
3,<unused0>브로콜리 컬리플라워 샐러드와 두유 요거트 소스<unused1><un...,두유에 플레인 요구르트와 레몬즙 설탕을 넣고 잘 섞어 두유 요구르트 소스를 만든다....,"[브로콜리, 컬리플라워, 샐러드, 적양파, 강낭콩, 건포도, 호두, 소금, 두유요거..."
4,<unused0>칠곡석류국수<unused1><unused2>소면$저염소금$식초$석류...,잣 아몬드 해바라기씨 호두 호박씨는 다진다. 끓는 물에 저염 소금과 식초를 넣은 후...,"[소면, 저염소금, 식초, 석류, 잣, 아몬드, 해바라기씨, 호두, 호박씨, 오이]"


미리 처리한 재료 리스트를 불러옵니다.

In [ ]:
with open('data/words_splitver.pickle', "rb") as fr:
    dic = pickle.load(fr)
dic

['고구마',
 '설탕',
 '찹쌀가루',
 '애호박',
 '표고버섯',
 '당근',
 '누룽지',
 '순두부',
 '달걀',
 '참기름',
 '소금',
 '참깨',
 '후추',
 '초밥밥',
 '새송이버섯',
 '새우',
 '홍피망',
 '청피망',
 '양파',
 '대두유',
 '식용유',
 '두부',
 '흰쌀',
 '현미쌀',
 '찹쌀',
 '실곤약',
 '나물콩나물',
 '고사리',
 '줄기',
 '플레인요거트',
 '곁들임새싹',
 '흑미밥쌀',
 '검은',
 '닭가슴살',
 '월계수',
 '나물숙주',
 '봉지',
 '토란대',
 '대파',
 '들기름',
 '실고추',
 '소금들깨',
 '고추기름',
 '저염간장',
 '취나물밥밥',
 '공기',
 '쇠고기',
 '취나물',
 '다진마늘',
 '간장',
 '매실즙',
 '된장',
 '콩가루',
 '멸치가루',
 '쌀뜨물',
 '호박',
 '호두',
 '해물장조갯살',
 '새우살',
 '오징어살',
 '감자',
 '청양고추',
 '마늘',
 '쌈장',
 '카놀라유',
 '고춧가루',
 '무초절임무',
 '식초',
 '은행',
 '단호박',
 '대추',
 '케일연잎도',
 '쪄진',
 '영양밥찹쌀',
 '흑미',
 '바지락',
 '강낭콩',
 '완두콩',
 '메주콩',
 '오색달걀',
 '가지',
 '소스다진',
 '잣가루',
 '연근',
 '새싹',
 '칵테일새우',
 '생크림',
 '무염버터',
 '고명시금치',
 '부추',
 '찜조선부추',
 '날콩가루',
 '요리당',
 '손질물',
 '구기자',
 '사과',
 '배추',
 '오이',
 '돌미나리',
 '쪽파',
 '방울토마토',
 '멸치액젓',
 '매실액',
 '통깨',
 '땅콩',
 '순두부사과',
 '소스순두부',
 '닭육수닭',
 '녹차가루',
 '백련초',
 '가루',
 '고명쇠고기',
 '멸치육수양파',
 '멸치',
 '밀가루',
 '콩나물',
 '저염된장',
 '된장국두부',
 '가닥',
 '다시마',
 '기둥',
 '국간장',
 '청경

평가 메트릭에 활용할 f1 score와 blue score를 정의합니다.

In [ ]:
from sklearn.metrics import f1_score
from konlpy.tag import Okt
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import brevity_penalty

def to_okt(output_text):
  okt = Okt()
  odf = okt.morphs(output_text)
  return odf

def get_output_ingr(list_dictionary, odf):
  out_ingr = list(set(list_dictionary) & set(odf))
  return out_ingr

def get_f1_score(actual_ingr_list, output_ingr_list):
  out_ingr = output_ingr_list
  actual = actual_ingr_list
  predict = out_ingr
  new_list = []
  for i in actual:
    if i not in new_list:
      new_list.append(i)
  new_list
  for i in predict:
    if i not in new_list:
      new_list.append(i)
  y_pred = []
  y_true = []
  for i in new_list:
    if i in predict:
      y_pred.append(1)
    else:
      y_pred.append(0)
    if i in actual:
      y_true.append(1)
    else:
      y_true.append(0)
  return f1_score(y_true,y_pred)

## 3. 모델 학습 및 검증

하이퍼파라미터를 미리 설정해두고, 평가 메트릭에 근거하여 Grid-Search를 진행합니다. 이때 평가 결과는 csv형태로 저장됩니다.

평가 메트릭을 초기화해줍니다.

In [ ]:
#기본세팅
filepath = "/content/drive/MyDrive/validation1.csv"
df = df #validation 데이터프레임
dic = dic #전체 식재료 목록

initialize_df = pd.DataFrame(data=['name','F1 Score','# of ingr', 'BLUE', 'BP']).transpose()
initialize_df.to_csv(filepath, mode = "w", header=False)

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

평가 대상 하이퍼파라미터를 지정합니다.

In [ ]:
from tqdm import notebook
from tqdm import tqdm

#hyperparameter 세팅
evaluation_size = 5
repetition_penalties=[1,1.1,1.2,1.3,1.5,1.75,1.9]
temperatures=[0.9] 
top_ks=[50]
top_ps=[0.92]

학습 및 검증을 진행합니다.

In [ ]:
okt = Okt()

for repetition_penalty in notebook.tqdm(repetition_penalties, desc='repetition_penalty'):
    for temperature in notebook.tqdm(temperatures, desc = 'temperature'):
        for top_k in notebook.tqdm(top_ks,  desc = 'top_k'):
            for top_p in notebook.tqdm(top_ps, desc = 'top_p'):
                evaluation_result = []

                for k in range(0,evaluation_size):
                    instance = []
                    input_text = df['input'][k]
                    input_ids = tokenizer.encode(input_text, return_tensors='pt')
                    with torch.no_grad():
                        generated_ids = model.generate(
                            input_ids,
                            do_sample=True,
                            min_length=40,
                            max_length=300,
                            repetition_penalty=repetition_penalty,
                            no_repeat_ngram_size=2,
                            temperature=temperature,
                            top_k=top_k,
                            top_p=top_p,
                        )
                    output_all = tokenizer.decode([el.item() for el in generated_ids[0]])
                    output = output_all.split('<unused4>')[1].split('<unused5')[0]
                    input_ingr = df['split'][k]
                    tokenized_output = okt.morphs(output)
                    output_ingr = list(set(dic) & set(tokenized_output))
                    instance.append(get_f1_score(input_ingr, output_ingr))
                    instance.append(len(output_ingr))
                    candidate = tokenized_output
                    reference = to_okt(df['output'][k])
                    instance.append(sentence_bleu(reference, candidate))
                    instance.append(brevity_penalty(len(reference), len(candidate)))
                    evaluation_result.append(instance)
                eval = pd.DataFrame(evaluation_result)
                eval.columns = ['F1 Score','# of ingr', 'BLUE', 'BP']
                evalmean = eval.mean()
                evalmean.loc['name'] = f"rep:{repetition_penalty}_temp:{temperature}_topk:{top_k}_topp:{top_p}"
                evaldf = pd.DataFrame(evalmean).transpose()
                evaldf = evaldf.reindex(columns=['name','F1 Score','# of ingr', 'BLUE', 'BP'])
                evaldf.to_csv(filepath, mode = "a", header=False)


best param은 github link를 참고해주세요.